![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("bank_marketing.csv")
#Vemos los tipos de datos de las columnas para saber a cuales les haremos cambios
#df.dtypes

#hacemos DataFrames de acuerdo a los archivos .csv
clients = df[["client_id","age","job","marital","education","credit_default","mortgage"]]
campaign = df[["client_id","number_contacts","contact_duration","previous_campaign_contacts","previous_outcome","campaign_outcome","month","day"]]
economics = df[["client_id","cons_price_idx","euribor_three_months"]]

#comenzamos a modificar clients
clients["job"] = clients["job"].str.replace(".","_")
clients["education"] = clients["education"].str.replace(".","_")
clients["education"] = clients["education"].replace("unknown",np.NaN)

#modificamos ambas columnas que requieren cambio de tipo de dato 
for columna in ["credit_default","mortgage"]:
    clients[columna] = clients[columna].map(lambda x: 1 if x== "yes" else 0)
    clients[columna] = clients[columna].astype(bool)

#comenzamos a modificar campaign
campaign["previous_outcome"] = campaign["previous_outcome"].map(lambda x: 1 if x == "success" else 0)
campaign["previous_outcome"] = campaign["previous_outcome"].astype(bool)
campaign["campaign_outcome"] = campaign["campaign_outcome"].map(lambda x: 1 if x == "yes" else 0)
campaign["campaign_outcome"] = campaign["campaign_outcome"].astype(bool)

#creamos la columna last_contact_date
#para ello creamos una columna con valor por default "2022" y cambiamos el tipo de dato de day
campaign["año"] = '2022'
campaign["day"] = campaign["day"].astype(str)
#ahora juntamos año, mes y dia en una misma columna
campaign["last_contact_date"] = campaign["año"] + "-" + campaign["month"] + "-" + campaign["day"]
#cambiamos el tipo de dato de "last_contact_date" a datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format = "%Y-%b-%d")

#borramos las columnas innecesarias de campaign
campaign.drop(columns =["month","day","año"],inplace = True)

#pasamos los dataFrames limpios a archivos .csv
clients.to_csv("client.csv",index=False)
campaign.to_csv("campaign.csv",index = False)
economics.to_csv("economics.csv",index = False)